In [57]:
import urllib, string, time
import urllib.parse
from astropy.coordinates import SkyCoord
from astropy import units as u
import pandas as pd
from astropy.time import Time
import numpy as np
from tabulate import tabulate
from astroquery.simbad import Simbad

from bokeh.plotting import output_notebook, figure, show
from bokeh.models import HoverTool, tools,ColumnDataSource
from bokeh.palettes import viridis, inferno
from bokeh.models import HoverTool, tools,ColumnDataSource, Whisker, ColorBar, LinearColorMapper

output_notebook()

Loading BokehJS ...

In [58]:
cvstable = pd.read_csv('harrisgcs.xls')


cvstable

,name,alt_name,ra,dec,helio_distance,reddening
0,NGC 6121,M 4,245.89675,-26.52575,2.2,0.35
1,NGC 6397,NaN,265.17537,-53.67433,2.3,0.18
2,NGC 6544,NaN,271.83575,-24.99733,3.0,0.76
3,NGC 6656,M 22,279.09975,-23.90475,3.2,0.34
4,NGC 6366,NaN,261.93433,-5.07986,3.5,0.71
...,...,...,...,...,...,...
152,NGC 2419,NaN,114.53529,38.88244,82.6,0.08
153,Eridanus,NaN,66.18542,-21.18694,90.1,0.02
154,Pal 3,NaN,151.38292,0.07167,92.5,0.04
155,Pal 4,NaN,172.32000,28.97358,108.7,0.01


In [73]:
with open('massive.md','w') as file:
    for name,ra,dec,dist,red in zip(cvstable.name,cvstable.ra,cvstable.dec
                                    ,cvstable.helio_distance,cvstable.reddening):
        file.write('\n')
        file.write(f'# {name} {dist} kpc reddening {red}\n')
        file.write('\n')

        namecluster = name
        try:
            result_table = Simbad.query_object(namecluster)
            table = result_table[0]

        except:
            continue
        #print(result_table)
        racluster = table['RA']
        deccluster = table['DEC']
        c = SkyCoord(f'{racluster} {deccluster}', unit=(u.hourangle, u.deg))


        ra = c.ra.deg
        dec= c.dec.deg
        radius = 3.0

        # create URL with desired search parameters

        url = "https://archive.stsci.edu/hst/search.php?"
        url = url + f"&RA={ra}"
        url = url + f"&DEC={dec}"
        url = url + f"&radius={radius}"
        url = url + "&outputformat=CSV"
        url = url + "&action=Search"
        #print(url)
        data = pd.read_csv(url,low_memory=False)
        #print(data)
        try:
            data = data.drop(data.index[0])
        except:
            continue
        data = data[data['Start Time'].apply(lambda x: isinstance(x, str))]

        #m4data
        #data['YearDay'] = data['Start Time'].map(lambda times: '.'.join(Time(times).yday.split(':',2)[:2]))

        #data['Week'] = data['Start Time'].map(lambda times: int(np.floor(float(Time(times).yday.split(':')[1])/7.)))
        #data2 = data[data['High-Level Science Products'] == '1']
        t =data.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','Proposal ID']).size().reset_index().rename(columns={0:'ocurrence'})

        data['year'] = data['Start Time'].map(lambda times: Time(times).yday.split(':')[0])
        datacleaner =data[(data['Target Name'] != 'DARK') & (data['Target Name'] != 'BIAS') & (data['Target Name'] != 'EARTH-CALIB')]


        for num,fil in enumerate(datacleaner['Filters/Gratings'].unique()):
            #print(f"# {fil}")
            file.write('\n')
            title=f"## {fil}\n"
            file.write('\n')
            file.write(title)
            filtertofilter = fil

            #datacleaner =data[(data['Target Name'] != 'DARK') & (data['Target Name'] != 'BIAS') & (data['Target Name'] != 'EARTH-CALIB')]
            dataf555w  = datacleaner[datacleaner['Filters/Gratings'] == filtertofilter]

            t2 =dataf555w.groupby(['Target Name','Filters/Gratings','Instrument','Exp Time','year']).size().reset_index().rename(columns={0:'ocurrence'})
            #t2 =data.groupby(['Target Name','Filters/Gratings','Instrument']).size().reset_index().rename(columns={0:'ocurrence'})

            t2 = t2.sort_values(by=['ocurrence'],ascending=False)
            t2['cum_sum'] = t2['ocurrence'].cumsum()
            
            file.write('\n')
            file.write(tabulate(t2.sort_values(by=['ocurrence'],ascending=False), tablefmt="pipe", headers="keys"))
            file.write('\n')





#t.sort_values(by=['ocurrence'],ascending=False)
    #print(tabulate(t.sort_values(by=['ocurrence'],ascending=False), tablefmt="pipe", headers="keys"))


/home/mmarcano/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '2MS-GC01': No known catalog could be found
  (error.line, error.msg))
/home/mmarcano/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): 'GLIMPSE01' this identifier has an incorrect format for catalog: 	GJ : Gliese - Jahreiss
  (error.line, error.msg))
/home/mmarcano/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '2MS-GC02': No known catalog could be found
  (error.line, error.msg))
/home/mmarcano/anaconda3/lib/python3.7/site-packages/astroquery/simbad/core.py:138: UserWarning: Warning: The script line numbe

In [60]:
url

'https://archive.stsci.edu/hst/search.php?&RA=245.89675&DEC=-26.52575&radius=3.0&outputformat=CSV&action=Search'

In [38]:
data

,no rows found


In [39]:
url

'https://archive.stsci.edu/hst/search.php?&RA=161.39369999999997&DEC=-49.61528&radius=3.0&outputformat=CSV&action=Search'

In [40]:
data = pd.read_csv(url)

In [41]:
data

,no rows found
